# Unballanced dataset model implementation

    
### Boosting alghoritms: 
    XGBoost
    AdaBoost
    Gradient Boosting Machine
    Optimized light gradient boosting 
    CatBoost

## Unballanced dataset Models

In [1]:
# Importing necessary libraries

import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('float_format', '{:f}'.format)

import matplotlib.pyplot as plt
import matplotlib.colors as colors  
import matplotlib.patches as mpatches
%matplotlib inline
import seaborn as sns

import time

import warnings
warnings.filterwarnings('ignore')


from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import collections
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, recall_score, precision_score

In [2]:
df = pd.read_csv('datasets/data1_unballanced.csv')

In [3]:
df.head()

,s_amount,s_time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,1.783274,-0.994983,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0
1,-0.269825,-0.994983,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0
2,4.983721,-0.994972,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0
3,1.418291,-0.994972,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0
4,0.670579,-0.994960,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0


In [4]:
# Spliting Training and testing data
X = df.drop('Class', axis=1)
y = df['Class']


In [5]:
sss = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)

In [6]:
for train_index, test_index in sss.split(X, y):
    print("Train:", train_index, "Test:", test_index)
    original_Xtrain, original_Xtest = X.iloc[train_index], X.iloc[test_index]
    original_ytrain, original_ytest = y.iloc[train_index], y.iloc[test_index]

# Check the Distribution of the labels
original_Xtrain = original_Xtrain.values
original_Xtest = original_Xtest.values
original_ytrain = original_ytrain.values
original_ytest = original_ytest.values

# Check to see if the distribution of the test and train labels is similar.
train_unique_label, train_counts_label = np.unique(original_ytrain, return_counts=True)
test_unique_label, test_counts_label = np.unique(original_ytest, return_counts=True)
print('-' * 100)

print('Label Distributions: \n')
print(train_counts_label/ len(original_ytrain))
print(test_counts_label/ len(original_ytest))

Train: [ 30473  30496  31002 ... 284804 284805 284806] Test: [    0     1     2 ... 57017 57018 57019]
Train: [     0      1      2 ... 284804 284805 284806] Test: [ 30473  30496  31002 ... 113964 113965 113966]
Train: [     0      1      2 ... 284804 284805 284806] Test: [ 81609  82400  83053 ... 170946 170947 170948]
Train: [     0      1      2 ... 284804 284805 284806] Test: [150654 150660 150661 ... 227866 227867 227868]
Train: [     0      1      2 ... 227866 227867 227868] Test: [212516 212644 213092 ... 284804 284805 284806]
----------------------------------------------------------------------------------------------------
Label Distributions: 

[0.99827076 0.00172924]
[0.99827952 0.00172048]


In [7]:
# setting up testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

## XGBoost

by including tweaking for max_depth and n_estimators alongside learning_rate, we'll set up a nested loop to iterate over combinations of these parameters. 

In [12]:
import time
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

# Define ranges for the parameters to test
learning_rates = [0.01, 0.1, 0.3]
max_depths = [3, 6, 9]
n_estimators = [20, 50, 100, 200, 500, 1000]

# Initialize a list to store the results
results = []

# Loop over the combinations of learning_rates, max_depths, and n_estimators
for lr in learning_rates:
    for depth in max_depths:
        for n_est in n_estimators:
            start_time = time.time()  # Start timing
            
            # Instantiate and fit the XGBoost classifier
            xgb_classifier = xgb.XGBClassifier(learning_rate=lr, max_depth=depth, n_estimators=n_est,
                                               use_label_encoder=False, eval_metric='logloss')
            xgb_classifier.fit(X_train, y_train)
            
            # Predict and evaluate on training data
            y_train_pred = xgb_classifier.predict(X_train)
            train_accuracy = accuracy_score(y_train, y_train_pred)
            train_f1 = f1_score(y_train, y_train_pred)
            train_recall = recall_score(y_train, y_train_pred)
            train_precision = precision_score(y_train, y_train_pred)
            
            # Predict and evaluate on testing data
            y_test_pred = xgb_classifier.predict(X_test)
            test_accuracy = accuracy_score(y_test, y_test_pred)
            test_f1 = f1_score(y_test, y_test_pred)
            test_recall = recall_score(y_test, y_test_pred)
            test_precision = precision_score(y_test, y_test_pred)
            
            # Calculate elapsed time
            end_time = time.time()
            elapsed_time = end_time - start_time
            
            # Append training and testing results separately, including parameter values and execution time
            params = f'LR: {lr}, Depth: {depth}, Estimators: {n_est}'
            
            print(params, elapsed_time)
            
            results.append({
                'Parameters': params,
                'Data Split': 'Training',
                'Accuracy': train_accuracy,
                'F1 Score': train_f1,
                'Recall': train_recall,
                'Precision': train_precision,
                'Execution Time (s)': elapsed_time
            })
            results.append({
                'Parameters': params,
                'Data Split': 'Testing',
                'Accuracy': test_accuracy,
                'F1 Score': test_f1,
                'Recall': test_recall,
                'Precision': test_precision,
                'Execution Time (s)': elapsed_time
            })

# Convert the list to a DataFrame
results_df = pd.DataFrame(results)

LR: 0.01, Depth: 3, Estimators: 20 0.9533159732818604
LR: 0.01, Depth: 3, Estimators: 50 1.0730090141296387
LR: 0.01, Depth: 3, Estimators: 100 1.5861845016479492
LR: 0.01, Depth: 3, Estimators: 200 2.5883305072784424
LR: 0.01, Depth: 3, Estimators: 500 5.996670961380005
LR: 0.01, Depth: 3, Estimators: 1000 12.715234518051147
LR: 0.01, Depth: 6, Estimators: 20 0.8032400608062744
LR: 0.01, Depth: 6, Estimators: 50 1.138197660446167
LR: 0.01, Depth: 6, Estimators: 100 1.787552833557129
LR: 0.01, Depth: 6, Estimators: 200 2.9922940731048584
LR: 0.01, Depth: 6, Estimators: 500 8.523376226425171
LR: 0.01, Depth: 6, Estimators: 1000 19.96651864051819
LR: 0.01, Depth: 9, Estimators: 20 1.0824594497680664
LR: 0.01, Depth: 9, Estimators: 50 1.5159637928009033
LR: 0.01, Depth: 9, Estimators: 100 1.8650438785552979
LR: 0.01, Depth: 9, Estimators: 200 3.9746532440185547
LR: 0.01, Depth: 9, Estimators: 500 9.209301233291626
LR: 0.01, Depth: 9, Estimators: 1000 21.599347591400146
LR: 0.1, Depth: 3, 

In [13]:
results_df.head(20)

,Parameters,Data Split,Accuracy,F1 Score,Recall,Precision,Execution Time (s)
0,"LR: 0.01, Depth: 3, Estimators: 20",Training,0.998271,0.000000,0.000000,0.000000,0.953316
1,"LR: 0.01, Depth: 3, Estimators: 20",Testing,0.998280,0.000000,0.000000,0.000000,0.953316
2,"LR: 0.01, Depth: 3, Estimators: 50",Training,0.998271,0.000000,0.000000,0.000000,1.073009
3,"LR: 0.01, Depth: 3, Estimators: 50",Testing,0.998280,0.000000,0.000000,0.000000,1.073009
4,"LR: 0.01, Depth: 3, Estimators: 100",Training,0.999364,0.798331,0.728426,0.883077,1.586185
5,"LR: 0.01, Depth: 3, Estimators: 100",Testing,0.999315,0.777143,0.693878,0.883117,1.586185
6,"LR: 0.01, Depth: 3, Estimators: 200",Training,0.999399,0.812585,0.753807,0.881306,2.588331
7,"LR: 0.01, Depth: 3, Estimators: 200",Testing,0.999386,0.806630,0.744898,0.879518,2.588331
8,"LR: 0.01, Depth: 3, Estimators: 500",Training,0.999565,0.865306,0.807107,0.932551,5.996671
9,"LR: 0.01, Depth: 3, Estimators: 500",Testing,0.999473,0.831461,0.755102,0.925000,5.996671


In [14]:
results_df.to_excel('XGBoost_results.xlsx', index=False)

## AdaBoost

To experiment with the AdaBoost algorithm and tweak its parameters, we can adjust the number of estimators (n_estimators) and the learning rate (learning_rate). These are two key parameters for AdaBoost that influence the performance and complexity of the resulting model. The n_estimators parameter controls the number of sequential models to be added to correct the errors of the previous models, while learning_rate scales the contribution of each model.

In [16]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

# Define ranges for the parameters to test
n_estimators_list = [10, 25, 50, 100] #, 500, 1000, 5000] #Too slow to run all. At 1000 took 4 hours and crashed.
learning_rates_list = [0.01, 0.1, 0.5, 1] #, 2]

# Initialize a list to store the results
results = []

# Loop over the combinations of n_estimators and learning_rates
for n_estimators in n_estimators_list:
    for learning_rate in learning_rates_list:
        start_time = time.time()  # Start timing
        
        # Instantiate and fit the AdaBoost classifier
        ada_classifier = AdaBoostClassifier(n_estimators=n_estimators, learning_rate=learning_rate)
        ada_classifier.fit(X_train, y_train)
        
        # Predict and evaluate on training data
        y_train_pred = ada_classifier.predict(X_train)
        train_accuracy = accuracy_score(y_train, y_train_pred)
        train_f1 = f1_score(y_train, y_train_pred)
        train_recall = recall_score(y_train, y_train_pred)
        train_precision = precision_score(y_train, y_train_pred)
        
        # Predict and evaluate on testing data
        y_test_pred = ada_classifier.predict(X_test)
        test_accuracy = accuracy_score(y_test, y_test_pred)
        test_f1 = f1_score(y_test, y_test_pred)
        test_recall = recall_score(y_test, y_test_pred)
        test_precision = precision_score(y_test, y_test_pred)
        
        # Calculate elapsed time
        end_time = time.time()
        elapsed_time = end_time - start_time
        
        # Append training and testing results separately, including parameter values and execution time
        params = f'Estimators: {n_estimators}, LR: {learning_rate}'
        print(params, elapsed_time)
        results.append({
            'Parameters': params,
            'Data Split': 'Training',
            'Accuracy': train_accuracy,
            'F1 Score': train_f1,
            'Recall': train_recall,
            'Precision': train_precision,
            'Execution Time (s)': elapsed_time
        })
        results.append({
            'Parameters': params,
            'Data Split': 'Testing',
            'Accuracy': test_accuracy,
            'F1 Score': test_f1,
            'Recall': test_recall,
            'Precision': test_precision,
            'Execution Time (s)': elapsed_time
        })

# Convert the list to a DataFrame
results_df = pd.DataFrame(results)

Estimators: 10, LR: 0.01 31.58523178100586
Estimators: 10, LR: 0.1 30.894355535507202
Estimators: 10, LR: 0.5 44.40583801269531
Estimators: 10, LR: 1 34.184561014175415
Estimators: 10, LR: 2 35.3106894493103
Estimators: 50, LR: 0.01 167.66188430786133
Estimators: 50, LR: 0.1 162.26059937477112
Estimators: 50, LR: 0.5 164.6594796180725
Estimators: 50, LR: 1 180.01729130744934
Estimators: 50, LR: 2 176.27559065818787
Estimators: 100, LR: 0.01 335.85931849479675
Estimators: 100, LR: 0.1 338.2623703479767
Estimators: 100, LR: 0.5 337.5833435058594
Estimators: 100, LR: 1 336.3496763706207
Estimators: 100, LR: 2 334.9484679698944
Estimators: 500, LR: 0.01 1651.756056547165
Estimators: 500, LR: 0.1 1727.608912229538
Estimators: 500, LR: 0.5 1687.1049993038177
Estimators: 500, LR: 1 1728.027375459671
Estimators: 500, LR: 2 8401.737571954727


KeyboardInterrupt: 

In [22]:
results_df.to_excel('AdaBoost_results.xlsx', index=False)

In [21]:
results_df = pd.DataFrame(results)
results_df.head(50)

,Parameters,Data Split,Accuracy,F1 Score,Recall,Precision,Execution Time (s)
0,"Estimators: 10, LR: 0.01",Training,0.999100,0.723347,0.680203,0.772334,31.585232
1,"Estimators: 10, LR: 0.01",Testing,0.999052,0.718750,0.704082,0.734043,31.585232
2,"Estimators: 10, LR: 0.1",Training,0.998977,0.623586,0.489848,0.857778,30.894356
3,"Estimators: 10, LR: 0.1",Testing,0.998947,0.615385,0.489796,0.827586,30.894356
4,"Estimators: 10, LR: 0.5",Training,0.999131,0.701807,0.591371,0.862963,44.405838
5,"Estimators: 10, LR: 0.5",Testing,0.999175,0.718563,0.612245,0.869565,44.405838
6,"Estimators: 10, LR: 1",Training,0.999149,0.729805,0.664975,0.808642,34.184561
7,"Estimators: 10, LR: 1",Testing,0.999105,0.727273,0.693878,0.764045,34.184561
8,"Estimators: 10, LR: 2",Training,0.000961,0.001229,0.355330,0.000615,35.310689
9,"Estimators: 10, LR: 2",Testing,0.000966,0.001053,0.306122,0.000528,35.310689


## AdaBoost with Decision Tree clacifier

AdaBoost, as implemented in scikit-learn (AdaBoostClassifier), doesn't directly accept a max_depth parameter because AdaBoost can be used with various types of base estimators, not just decision trees. This is a frequent way of using AdaBoost.

In [ ]:
import time
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

# Define ranges for the parameters to test
n_estimators_list = [10, 50, 100, 500, 1000, 5000]
learning_rates_list = [0.1, 0.5, 1]
max_depths = [3, 6, 9]

# Initialize a list to store the results
results = []

# Loop over the combinations of n_estimators, learning_rates, and max_depths
for n_estimators in n_estimators_list:
    for learning_rate in learning_rates_list:
        for max_depth in max_depths:
            start_time = time.time()  # Start timing
            
            # Create a DecisionTreeClassifier with the specified max_depth
            base_estimator = DecisionTreeClassifier(max_depth=max_depth)
            
            # Instantiate and fit the AdaBoost classifier with the DecisionTree base estimator
            ada_classifier = AdaBoostClassifier(
                base_estimator=base_estimator,
                n_estimators=n_estimators,
                learning_rate=learning_rate
            )
            ada_classifier.fit(X_train, y_train)
            
            # Predict and evaluate on training data
            y_train_pred = ada_classifier.predict(X_train)
            train_accuracy = accuracy_score(y_train, y_train_pred)
            train_f1 = f1_score(y_train, y_train_pred)
            train_recall = recall_score(y_train, y_train_pred)
            train_precision = precision_score(y_train, y_train_pred)
            
            # Predict and evaluate on testing data
            y_test_pred = ada_classifier.predict(X_test)
            test_accuracy = accuracy_score(y_test, y_test_pred)
            test_f1 = f1_score(y_test, y_test_pred)
            test_recall = recall_score(y_test, y_test_pred)
            test_precision = precision_score(y_test, y_test_pred)
            
            # Calculate elapsed time
            end_time = time.time()
            elapsed_time = end_time - start_time
            
            # Append training and testing results separately, including parameter values and execution time
            params = f'Estimators: {n_estimators}, LR: {learning_rate}, Depth: {max_depth}'
            print(params, elapsed_time)
            results.append({
                'Parameters': params,
                'Data Split': 'Training',
                'Accuracy': train_accuracy,
                'F1 Score': train_f1,
                'Recall': train_recall,
                'Precision': train_precision,
                'Execution Time (s)': elapsed_time
            })
            results.append({
                'Parameters': params,
                'Data Split': 'Testing',
                'Accuracy': test_accuracy,
                'F1 Score': test_f1,
                'Recall': test_recall,
                'Precision': test_precision,
                'Execution Time (s)': elapsed_time
            })

# Convert the list to a DataFrame
results_df = pd.DataFrame(results)


In [ ]:
results_df.to_excel('AdaboostDT_results.xlsx', index=False)

In [ ]:
### Was run in different instance on another similar computer.

## Gradient Boosting Machine

To perform a similar parameter tweaking experiment with a Gradient Boosting Machine (GBM) using scikit-learn's GradientBoostingClassifier, we can iterate over combinations of n_estimators, learning_rate, and max_depth. These parameters are analogous to those adjusted in the AdaBoost and XGBoost, serving similar purposes:

n_estimators: controls the number of sequential trees to be modeled.
learning_rate: scales the contribution of each tree.
max_depth: sets the maximum depth of each tree.

In [23]:
import time
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

# Define ranges for the parameters to test
n_estimators_list = [10, 50, 100]
learning_rates_list = [0.01, 0.1, 0.3]
max_depths = [3, 6, 9]

# Initialize a list to store the results
results = []

# Loop over the combinations of n_estimators, learning_rates, and max_depths
for n_estimators in n_estimators_list:
    for learning_rate in learning_rates_list:
        for max_depth in max_depths:
            start_time = time.time()  # Start timing
            
            # Instantiate and fit the GradientBoostingClassifier
            gbm_classifier = GradientBoostingClassifier(
                n_estimators=n_estimators,
                learning_rate=learning_rate,
                max_depth=max_depth
            )
            gbm_classifier.fit(X_train, y_train)
            
            # Predict and evaluate on training data
            y_train_pred = gbm_classifier.predict(X_train)
            train_accuracy = accuracy_score(y_train, y_train_pred)
            train_f1 = f1_score(y_train, y_train_pred)
            train_recall = recall_score(y_train, y_train_pred)
            train_precision = precision_score(y_train, y_train_pred)
            
            # Predict and evaluate on testing data
            y_test_pred = gbm_classifier.predict(X_test)
            test_accuracy = accuracy_score(y_test, y_test_pred)
            test_f1 = f1_score(y_test, y_test_pred)
            test_recall = recall_score(y_test, y_test_pred)
            test_precision = precision_score(y_test, y_test_pred)
            
            # Calculate elapsed time
            end_time = time.time()
            elapsed_time = end_time - start_time
            
            # Append training and testing results separately, including parameter values and execution time
            params = f'Estimators: {n_estimators}, LR: {learning_rate}, Depth: {max_depth}'
            print(params, elapsed_time)
            results.append({
                'Parameters': params,
                'Data Split': 'Training',
                'Accuracy': train_accuracy,
                'F1 Score': train_f1,
                'Recall': train_recall,
                'Precision': train_precision,
                'Execution Time (s)': elapsed_time
            })
            results.append({
                'Parameters': params,
                'Data Split': 'Testing',
                'Accuracy': test_accuracy,
                'F1 Score': test_f1,
                'Recall': test_recall,
                'Precision': test_precision,
                'Execution Time (s)': elapsed_time
            })

# Convert the list to a DataFrame
results_df = pd.DataFrame(results)



Estimators: 10, LR: 0.01, Depth: 3 85.18600678443909
Estimators: 10, LR: 0.01, Depth: 6 175.2360532283783
Estimators: 10, LR: 0.01, Depth: 9 275.31504702568054
Estimators: 10, LR: 0.1, Depth: 3 94.39143300056458
Estimators: 10, LR: 0.1, Depth: 6 188.76076078414917
Estimators: 10, LR: 0.1, Depth: 9 274.9938325881958
Estimators: 10, LR: 0.3, Depth: 3 96.43201422691345
Estimators: 10, LR: 0.3, Depth: 6 187.4304609298706
Estimators: 10, LR: 0.3, Depth: 9 298.10074400901794
Estimators: 50, LR: 0.01, Depth: 3 469.042222738266
Estimators: 50, LR: 0.01, Depth: 6 857.1180782318115
Estimators: 50, LR: 0.01, Depth: 9 1464.6021127700806
Estimators: 50, LR: 0.1, Depth: 3 425.05847239494324
Estimators: 50, LR: 0.1, Depth: 6 829.3951048851013
Estimators: 50, LR: 0.1, Depth: 9 1250.7942807674408
Estimators: 50, LR: 0.3, Depth: 3 418.06535363197327
Estimators: 50, LR: 0.3, Depth: 6 840.3917133808136
Estimators: 50, LR: 0.3, Depth: 9 1228.5074632167816
Estimators: 100, LR: 0.01, Depth: 3 863.32825684547

In [24]:
results_df.head()
results_df.to_excel('GBM_results.xlsx', index=False)